## Diagonalization of matrices

Created on Wed Feb 12 10:44:59 2020
@author: German Sinuco

Skeleton modified from 

https://www.tensorflow.org/tutorials/customization/custom_training 
https://www.tensorflow.org/tutorials/customization/custom_training_walkthrough

The original search_diagonalisation is in the script with the same name. Here I cut the printing instructions. The aim here is to parametrise the elements of the unitary matrix and adjust those parameters.
Training of an RBM parametrization of the unitary matrix that diagonalises the 2x2 real, and symmetric HAMILTONIAN:

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import numpy as np
import math as m
from scipy.stats import unitary_group
#tf.compat.v1.disable_eager_execution()

## Diagonalisation of a 2x2 matrix

Here we use `Tensor`, `Variable`, and `GradientTape` to build and train a model to diagonalise a 2x2 matix. This involves a few steps:

1. Define the model: define the matrix, H, and the unitary transformation that diagonalises it, U
2. Define a loss function: as the sum of the square of the offdiagonnal elements. Include a Lagrange multiplier to preserve unitarity.
4. Loop through the loss function and use an "optimizer" to adjust the variables to find U.

Here, the matrix is the Hamiltonian of a qubit in a magnetic field:

H =  delta S_z + Omega/2 Sx

The unitary transformation U, leads to the diagonal form:

H = U^dagger H U

and we parametrise the elements of the matrix:

U_ji = sqrt(P(x)_(b,c,W)/Z)

with:
    
x = j,i
 
and
    
P(x)_(b,c,W) = exp(b_ji . x) Prod_l=1^M 2 x cosh(c_l + W_{x,l} . x)

Z = sum_x P(x)_(b,c,W)

### Define the model

The model defines the Hamiltonian and the self call

In [27]:
class Model(object):
  def __init__(self):

    # Hamiltonian parameters
    self.delta = 1.00
    self.Omega  = 0.10

    # Initialize the spin value and number of floquet channels
    self.hidden = 2  # hidden neurons
    #self.S   = 4      # spin 3.2. Hilbert space dimension
    self.S   = 2     # spin 1/2. Hilbert space dimension
    self.N   = 0      # Number of positive Floquet manifolds
    self.dim = self.S*(2*self.N+1) # Dimension of the extended floquet space
    
    uf_      = tf.random.stateless_uniform([self.dim,self.dim],seed=[2,1],dtype=tf.float32,minval=0.0,maxval=1.0)      
    s,u,v    = tf.linalg.svd(uf_, full_matrices=True)
    uf_      = u
    
    # Declaring training variables
    self.W   = tf.Variable(tf.random.stateless_uniform([self.hidden,self.dim*self.dim,2],
                                                       seed=[1,1],dtype=tf.float32,minval=0.0,maxval=1.0),trainable=True) 
    self.b   = tf.Variable(tf.random.stateless_uniform([self.dim*self.dim,2], 
                                                       seed=[1,1],dtype=tf.float32,minval=0.0,maxval=1.0),trainable=True)
    self.c   = tf.Variable(tf.random.stateless_uniform([self.hidden],                    
                                                       seed=[1,1],dtype=tf.float32,minval=0.0,maxval=1.0),trainable=True)
    self.UF  = tf.Variable(uf_,dtype = tf.float32,trainable = False)  # ext. micromotion operator
    #print(uf_)
    self.x = tf.Variable([[0.0,0.0]],dtype=tf.float32)
    for i in range(1,self.dim+1):        
        for j in range(1,self.dim+1):
            y = [[i-2.5,j-2.5]]
            self.x = tf.concat([self.x, y], 0) 
    #print(self.x)
    #print(self.b)
    #print(self.c)
    #print(self.W)
    #print(tf.multiply(x[1:17],self.W[1]))
    #P(x)(b,c,W) = exp(bji . x) Prod_l=1^M 2 x cosh(c_l + W_{x,l} . x)
    WX = [tf.reduce_sum(tf.multiply(self.x[1:17],self.W[0]),1)+self.c[0]]
    for j in range(1,self.hidden):
        y = tf.reduce_sum(tf.multiply(self.x[1:17],self.W[j]),1)+self.c[j]
        WX = tf.concat([WX, [y]], 0) 
    
    
    UF = tf.multiply(tf.reduce_prod(tf.math.cosh(WX),0),tf.transpose(tf.reduce_sum(tf.multiply(self.x[1:17],self.b),1)))
    UF = tf.reshape(UF,[self.dim,self.dim])
    UF =  normalisation(UF)
    UF = Gram_Schmidt(UF)

    self.UF = UF
    print(UF)
    #print(Gram_Schmidt(UF))
    #print(tf.tensordot(self.W,x[1],[[2],[1]]))
    #print(self.UF)
    #print(tf.multiply(x,self.b))
    #print(tf.transpose(tf.reduce_sum(tf.multiply(x,self.b),1)))
    
    if self.S == 2:
        # spin 1/2 
        Identity   =     tf.constant([[1.0,0.0],[ 0.0, 1.0]],dtype = tf.float32)
        Sx         = 0.5*tf.constant([[0.0,1.0],[ 1.0, 0.0]],dtype = tf.float32)
        Sz         = 0.5*tf.constant([[1.0,0.0],[ 0.0,-1.0]],dtype = tf.float32)
    else:
        if self.S == 4:
        # spin 3/2
            Identity   =     tf.constant([[1.0,0.0,0.0,0.0],
                                  [0.0,1.0,0.0,0.0],
                                  [0.0,0.0,1.0,0.0],
                                  [0.0,0.0,0.0,1.0]],dtype = tf.float32)
            Sx         = 0.5*tf.constant([[0.0,         np.sqrt(3.0),0.0,          0.0],
                                  [np.sqrt(3.0),0.0,         np.sqrt(4.0), 0.0],
                                  [0.0,         np.sqrt(4.0),0.0,          np.sqrt(4.0)],
                                  [0.0,         0.0,         np.sqrt(3.0), 0.0]],dtype = tf.float32)
            Sz         = 0.5*tf.constant([[3.0,0.0, 0.0, 0.0],
                                  [0.0,1.0, 0.0, 0.0],
                                  [0.0,0.0,-1.0, 0.0],
                                  [0.0,0.0, 0.0,-3.0]],dtype = tf.float32)

    
    
    self.H_TLS = tf.Variable(self.delta*Sz+0.5*self.Omega*Sx,shape=(self.dim,self.dim),dtype = tf.float32,trainable = False)  # ext. Hamiltonian
        
    self.trainable_variables = [self.W,self.b,self.c]#[self.UF,self.W,self.b,self.c]
    #self.trainable_variables = self.UF
    
  def getH(self):
    return self.H_TLS
    
  def __call__(self):     
    #WX = [tf.reduce_sum(tf.multiply(self.x[1:17],self.W[0]),1)+self.c[0]]
    #for j in range(1,self.hidden):
    #    y = tf.reduce_sum(tf.multiply(self.x[1:17],self.W[j]),1)+self.c[j]
    #    WX = tf.concat([WX, [y]], 0) 
    
    
    #UF = tf.sqrt(tf.multiply(tf.reduce_prod(tf.math.cosh(WX),0),tf.transpose(tf.reduce_sum(tf.multiply(self.x[1:17],self.b),1))))
    #UF = tf.reshape(self.UF,[self.dim,self.dim])
    #self.UF = Gram_Schmidt(self.UF)
   # print(self.UF)
    
    return self.H_TLS
  

In [28]:
def normalisation(U_):
    # U_  (in) original matrix
    #     (out)  matrix with normalised vectors
    normaU_ = tf.sqrt(tf.math.reduce_sum(tf.multiply(U_,U_,1),axis=0))
    U_ = tf.math.truediv(U_,normaU_)
    return U_

def Gram_Schmidt(U_):
    U_ = normalisation(U_)
    #print(U_)
    Y_ = []
    #Y  = tf.Variable(np.empty([tf.shape(U_).numpy()[0],tf.shape(U_).numpy()[0]],dtype=np.float32))
    #Y.assign(U_)
    for i in range(0,tf.shape(U_).numpy()[0]):
        
        temp_vec = U_[:,i]
        #print(temp_vec)
        for inY in Y_:
            
            norminY    = tf.math.reduce_sum(tf.multiply(inY,inY,0)).numpy()
            projec_vec = tf.math.multiply(tf.math.reduce_sum(tf.multiply(U_[:,i],inY,0))/norminY,inY)
            #print(i,projec_vec,norminY)
            temp_vec   = tf.math.subtract(temp_vec,projec_vec)
            
        temp_vec = normalisation(temp_vec)     
        Y_.append(temp_vec)    

    Y_ = np.transpose(np.stack(Y_))
    U_ = Y_#U_.assign(Y_)

    # maybe it's worth to lose the conribution to the gradient from the GramSchmidt calculation.
    # but getting an orthonormal basis after applying the gradients it's needed to obtain 
    # UF
    # maybe replace UF = UF + Y - UF
    # o hacerlo sin eager mode
    return U_

def Gram_Schmidt_eltos(U_):
    D = tf.shape(U_).numpy()[0]
    Y = []
    for k in range(0,D):
        for i in range(0,D):

            g = 0
            #print(i,k)
            for j in range(0,k):
            #    print(j)
                for l in range(0,D):
                    g += U_[l,j]*U_[l,k]*U_[i,j]
                   # print(g)
           # print(i,k,g)
            Y.append(U_[i,k]-g)
            
        
    return Y


def train(model,learning_rate):
  with tf.GradientTape() as t:
    current_loss = loss(model)
  dU = t.gradient(current_loss, model.trainable_variables)
  model.UF.assign_sub(learning_rate*dU)
    #where should I obtain the unitary matrix? after this step?
    # should we apply a constrain as the one in the finantial applications?

### Define a loss function and its gradient

A loss function measures how well the output of a model for a given input matches the target output. The goal is to minimize this difference during training. Let's use the standard L2 loss, also known as the least square errors:

In [49]:
# 3e. Loss function := Use U^dagger H U, sum over the columns, take the difference with the diagonal, 
#     the loss function is the summ of the square of these differences. 

def loss(model):
  # define the loss function explicitly including the training variables: self.W, self.b
  # model.UF is a function of self.W,self.b,self.c

  WX = [tf.reduce_sum(tf.multiply(model.x[1:17],model.W[0]),1)+model.c[0]]
  for j in range(1,model.hidden):
        y = tf.reduce_sum(tf.multiply(model.x[1:17],model.W[j]),1)+model.c[j]
        WX = tf.concat([WX, [y]], 0) 
    
  UF = tf.multiply(tf.reduce_prod(tf.math.cosh(WX),0),
                           tf.transpose(tf.reduce_sum(tf.multiply(model.x[1:17],model.b),1)))
  UF = tf.reshape(UF,[model.dim,model.dim])
  UF = normalisation(UF)

  #U = UF
  
  #i = 1
  #temp_vec = U[:,1]
  #inY = U[:,0]
  #norminY    = tf.math.reduce_sum(tf.multiply(inY,inY,0)).numpy()
  #projec_vec = tf.math.multiply(tf.math.reduce_sum(tf.multiply(U[:,1],inY,0))/norminY,inY)
  #print(i,projec_vec,norminY)    
    
  #proj = tf.tensordot(UF[:,0],UF,axes=1)
  #print(proj)
  #U = tf.math.subtract(UF,tf.math.multiply(proj,UF))
  #print(U) 
  #U = normalisation(U)
  #proj = tf.tensordot(U[:,1],UF,axes=1)
  #print(proj)
  #U = tf.math.subtract(UF,tf.math.multiply(proj,UF))
  #print(U) 
  #proj = tf.tensordot(U[:,2],UF,axes=1)
  #print(proj)
  #U = tf.math.subtract(UF,tf.math.multiply(proj,UF))
  #print(U) 
  #proj = tf.tensordot(U[:,3],UF,axes=1)
  #print(proj)
  #U = tf.math.subtract(UF,tf.math.multiply(proj,UF))
  #print(U) 

  #U = U_[:,1].assign(U__[:,1])
 
  #print(UF) 
  #print(tf.transpose(UF)@UF)
  #print(tf.transpose(UF)@model.H_TLS@UF)
  U_ = tf.abs(tf.transpose(UF)@model.H_TLS@UF)
  U_diag = tf.linalg.tensor_diag_part(U_)  
  dotProd = tf.math.reduce_sum(U_,axis=1,)
  residual = tf.math.reduce_sum(tf.pow((U_diag-dotProd),2),0)
  #print(residual)  
  return residual

# This is the gradient of the loss function. required for keras optimisers
def grad(model):
  with tf.GradientTape() as tape:
    loss_value = loss(model)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)


optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

model = Model()
model()
# Collect the history of W-values and b-values to plot later
#Us = [], []
#learning_rate =0.01
epochs = range(64)
for i in epochs:
    loss_value, grads = grad(model)
    #print(model.trainable_variables)
    #model.UF.assign_sub(learning_rate*grads)
    #model.UF.assign(Gram_Schmidt(model.UF))
    #print(model.UF)

    #print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(),
    #                                      loss_value.numpy()))

    #print(model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    #model()
    #print("Step: {},         Loss: {}".format(optimizer.iterations.numpy(),
    #                                      loss(model).numpy()))
    #model.UF = Gram_Schmidt(model.UF)
    #model.UF.assign(Gram_Schmidt(model.UF))
    #model()
    #Gram_Schmidt(model.UF)


#print(model.UF)
#print(current_loss.numpy())
#print(model.UF.numpy())
#print(tf.transpose(model.UF)@(model.H_TLS@model.UF))
 
 

[[-0.7741894   0.6329542 ]
 [-0.63295406 -0.7741893 ]]


In [6]:
epochs = range(1)
#print(model.x)
current_loss = loss(model)
print(current_loss.numpy())
#print(grad(model))
#print(model.UF.numpy())
#print(tf.transpose(model.UF)@(model.H_TLS@model.UF))

learning_rate = 0.01
for i in epochs:
    with tf.GradientTape() as t:
        current_loss = loss(model)
        dU = t.gradient(current_loss, model.trainable_variables)
    print(dU)    
    #model.UF.assign_sub(learning_rate*dU[0])
    #model.UF.assign(Gram_Schmidt(model.UF))

    #print(model.UF.numpy())
    current_loss = loss(model)
    print(current_loss.numpy())
#print(current_loss.numpy())
#print(model.UF.numpy())
print(tf.transpose(model.UF)@(model.H_TLS@model.UF))

1.5030012e-06
[<tf.Tensor: id=946, shape=(2, 4, 2), dtype=float32, numpy=
array([[[ 1.9080981e-05,  1.9080981e-05],
        [ 4.2417123e-05,  1.4139041e-05],
        [-1.0918669e-05, -3.2756008e-05],
        [-1.5620561e-05, -1.5620561e-05]],

       [[ 6.6033404e-05,  6.6033404e-05],
        [-4.4057238e-06, -1.4685746e-06],
        [-4.2026668e-06, -1.2608000e-05],
        [-4.1771282e-06, -4.1771282e-06]]], dtype=float32)>, <tf.Tensor: id=893, shape=(4, 2), dtype=float32, numpy=
array([[ 2.0930612e-04,  2.0930612e-04],
       [ 1.0222221e-04,  3.4074070e-05],
       [-4.4141598e-05, -1.3242479e-04],
       [-3.0317911e-05, -3.0317911e-05]], dtype=float32)>, <tf.Tensor: id=947, shape=(2,), dtype=float32, numpy=array([ 1.2079727e-05, -2.4325533e-05], dtype=float32)>]
1.5030012e-06
tf.Tensor(
[[ 0.12387051 -0.4850579 ]
 [-0.4850579  -0.12387051]], shape=(2, 2), dtype=float32)


In [7]:
t1 = [[1,4]] 
t2 = [[7,10]] 
tf.concat([t1, t2], 0) 


<tf.Tensor: id=1043, shape=(2, 2), dtype=int32, numpy=
array([[ 1,  4],
       [ 7, 10]], dtype=int32)>

### Exact solution

First, build the exact unitary transformation and perturb it

In [8]:
Sx    = 0.5*tf.constant([[0,1],[ 1, 0]],dtype = tf.float32)
Sz    = 0.5*tf.constant([[1,0],[ 0,-1]],dtype = tf.float32)   
delta = 1.0
Omega = 0.1

H_TLS = delta*Sz+0.5*Omega*Sx
theta = 0.5*np.arctan(0.25*Omega/(-0.5*delta))
U     = [[np.cos(theta),np.sin(theta)],[-np.sin(theta),np.cos(theta)]]
U     = tf.convert_to_tensor(U, np.float32)
noise = tf.random.stateless_uniform([2,2],seed=[1,1],dtype=tf.float32,minval=0.0,maxval=0.1)      
U     = U #+ noise

print(H_TLS)
print(U)

print(H_TLS@U)

print(tf.transpose(U)@(H_TLS@U))



tf.Tensor(
[[ 0.5    0.025]
 [ 0.025 -0.5  ]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[ 0.999688  -0.0249766]
 [ 0.0249766  0.999688 ]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[ 0.50046843  0.0125039 ]
 [ 0.0125039  -0.50046843]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[ 0.50062466  0.        ]
 [ 0.         -0.50062466]], shape=(2, 2), dtype=float32)


### Define a training loop

With the network and training data, train the model using [gradient descent](https://en.wikipedia.org/wiki/Gradient_descent) to update the matrix 'U' to reduce the loss. There are many variants of the gradient descent scheme that are captured in `tf.train.Optimizer`—our recommended implementation. But in the spirit of building from first principles, here you will implement the basic math yourself with the help of `tf.GradientTape` for automatic differentiation and `tf.assign_sub` for decrementing a value (which combines `tf.assign` and `tf.sub`):

In [9]:
def normalisation(U_):
    # U_  (in) original matrix
    #     (out)  matrix with normalised vectors
    normaU_ = tf.sqrt(tf.math.reduce_sum(tf.multiply(U_,U_,1),axis=0))
    U_ = tf.math.truediv(U_,normaU_)
    return U_

def Gram_Schmidt(U_):
    U_ = normalisation(U_)
    Y_ = []
    Y  = tf.Variable(np.empty([tf.shape(U_).numpy()[0],tf.shape(U_).numpy()[0]],dtype=np.float32))
    Y.assign(U_)
    for i in range(0,tf.shape(U_).numpy()[0]):
        
        temp_vec = U_[:,i]
        
        for inY in range(0,i):#Y_:
            
            norminY    = tf.math.reduce_sum(tf.multiply(inY,inY,0)).numpy()
            projec_vec = tf.math.multiply(tf.math.reduce_sum(tf.multiply(U_[:,i],inY,0))/norminY,inY)
            temp_vec   = tf.math.subtract(temp_vec,projec_vec)
              
        temp_vec = normalisation(temp_vec)     
        Y_.append(temp_vec)    

    Y_ = np.transpose(np.stack(Y_))
    #U_.assign(Y_)

    # maybe it's worth to lose the conribution to the gradient from the GramSchmidt calculation.
    # but getting an orthonormal basis after applying the gradients it's needed to obtain 
    # UF
    # maybe replace UF = UF + Y - UF
    # o hacerlo sin eager mode
    return U_

def train(model,learning_rate):
  with tf.GradientTape() as t:
    current_loss = loss(model)
  dU = t.gradient(current_loss, model.trainable_variables)
  model.UF.assign_sub(learning_rate*dU)
    #where should I obtain the unitary matrix? after this step?
    # should we apply a constrain as the one in the finantial applications?

Finally, let's repeatedly run through the training data and see how `W` and `b` evolve.

In [10]:
model = Model()

# Collect the history of W-values and b-values to plot later
Us = [], []

epochs = range(1)
current_loss = loss(model)
print(current_loss.numpy())

#for epoch in epochs:
  #Us.append(model.UF.numpy())  
  #print(model.UF.numpy())
  #current_loss = loss(model)
  #print(current_loss.numpy())
  #train(model, learning_rate=1.0)

#print(current_loss.numpy())
#print(model.UF.numpy())
  

1.5043587e-06


In [11]:
tf.Variable(np.empty([2,2],dtype=np.float32))

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1.e-45, 0.e+00],
       [0.e+00, 0.e+00]], dtype=float32)>